In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
dataset = pd.read_csv('../../data/funda_train.csv')

In [3]:
dataset['year_month'] = dataset['transacted_date'].str[0:7]

In [4]:
drop_features = ['installment_term', 'region', 'type_of_business','transacted_date','card_company','transacted_time']
dataset = dataset.drop(drop_features, axis=1)

In [5]:
subset = dataset[['store_id','amount','year_month','card_id']]
subset

refund = subset[subset['amount'] < 0]
refund

,store_id,amount,year_month,card_id
41,0,-8571.428571,2016-06,40
347,0,-1857.142857,2016-08,285
731,0,-2000.000000,2016-10,473
831,0,-85.714286,2016-11,230
944,0,-57.142857,2016-11,138
...,...,...,...,...
6556242,2136,-13428.571429,2019-02,4663626
6556448,2136,-6928.571429,2019-02,4663760
6556485,2136,-5571.428571,2019-02,4663779
6556489,2136,-8571.428571,2019-02,4663780


In [6]:
dataset.iloc[refund.index[0] -1 : refund.index[0] + 1]

,store_id,card_id,amount,year_month
40,0,40,8571.428571,2016-06
41,0,40,-8571.428571,2016-06


In [7]:
refund_case_1 = []
for i in tqdm(range(len(refund.index))):
    subset = dataset.iloc[refund.index[i]-1 : refund.index[i] + 1]
    if subset['amount'].sum() == 0:
        refund_case_1.append(refund.index[i]) # 환불 건의 index 를 추출합니다. index 41과 같은 건을 모두 추출
    else:
        pass

100%|██████████| 73100/73100 [00:24<00:00, 3042.42it/s]


In [8]:
refund_case_2 = []
for i in tqdm(range(len(refund.index))):
    subset = dataset.iloc[refund.index[i]-1 : refund.index[i] + 1]
    if subset['amount'].sum() == 0:
        refund_case_2.append(refund.index[i]-1)  # 환불 전 결제 건의 index 를 추출합니다. index 40과 같은 건을 모두 추출
    else:
        pass

100%|██████████| 73100/73100 [00:16<00:00, 4536.85it/s]


In [9]:
print("refund_case_1 : {refund} refund_case_2 : {payment}".format(refund = len(refund_case_1), payment = len(refund_case_2)))
if len(refund_case_1) == len(refund_case_2):
    print("두 케이스 동일")

refund_case_1 : 55347 refund_case_2 : 55347
두 케이스 동일


In [10]:
sub_refund = refund.drop(index = refund_case_1, axis = 0)

print("기존 환불 건수 : {before} 드랍 후 환불 건수 : {after}".format(before = len(refund), after = len(sub_refund)))

기존 환불 건수 : 73100 드랍 후 환불 건수 : 17753


In [11]:
sub_dataset = dataset.drop(index = refund_case_2, axis = 0)

In [12]:
refund_case_3 = []
for i in tqdm(range(len(sub_refund.index))):
    subset = sub_dataset.loc[(sub_dataset['amount'] == abs(sub_refund.iloc[i].amount)) & (sub_dataset['card_id'] == (sub_refund.iloc[i].card_id))] # 환불 전 결제 케이스
    refund_case_3.append(subset.index.min())

100%|██████████| 17753/17753 [07:15<00:00, 40.80it/s]


In [13]:
refund_case_3 = [x for x in refund_case_3 if np.isnan(x) == False]

- -- -------------

In [14]:
refund_case_4 = list(sub_refund.index)

In [15]:
print("refund_case_3 : {before} refund_case_4 : {after}".format(before = len(refund_case_3), after = len(refund_case_4))) # 환불 케이스

refund_case_3 : 17657 refund_case_4 : 17753


In [16]:
len(dataset) - 73100*2 # 우리가 원하는 결과

6410413

In [17]:
dataset = dataset.drop(index = refund_case_1, axis = 0)
len(dataset)

6501266

In [18]:
dataset = dataset.drop(index = refund_case_2, axis = 0)
len(dataset)

6445919

In [19]:
dataset = dataset.drop(index = refund_case_3, axis = 0)
len(dataset)

6428813

In [20]:
dataset = dataset.drop(index = refund_case_4, axis = 0)
len(dataset)

6411060

In [22]:
print("해결하지 못한 환불 건수 : {case}".format(case = len(dataset) - 6410413))

해결하지 못한 환불 건수 : 647
